# Selenium을 이용한 동적 크롤링

In [13]:
import requests, time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [14]:
from bs4 import BeautifulSoup

url = 'https://sports.news.naver.com/kbaseball/news/index.nhn?isphoto=N'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

# BeautifulSoup으로 크롤링을 하려고 하였지만 본문은 크롤링이 안됨

### 야구 > 최신뉴스

In [15]:
driver = webdriver.Chrome('chromedriver')
url = 'https://sports.news.naver.com/kbaseball/news/index.nhn?isphoto=N'
driver.get(url)

In [16]:
# 최신 뉴스 리스트의 맨 마지막 페이지
while True:
    page = driver.find_element_by_id('_pageList')
    try:
        next = page.find_element_by_class_name('next')
        if next:
            next.click()
    except:
        break;

In [17]:
page = driver.find_element_by_id('_pageList')
try:
    atags = page.find_elements_by_tag_name('a')
    numPage = int(atags[-1].text)
except:
    numPage = 1
numPage

1

In [18]:
news = driver.find_element_by_id('_newsList')
lis = news.find_elements_by_tag_name('li')
len(lis)

7

In [19]:
for li in lis:
    text = li.find_element_by_class_name('title')
    title = text.find_element_by_tag_name('span').text
    print(title)

김경문 감독 “신에게는 154명의 태극전사가 있나이다”
“프로서 해 볼 만하단 자신감 챙겼죠”
"반갑다 친구야" 꿈의 구장에서 추신수와 이대호는 웃었다
'첫 안타 치자 동료들 환호' 쑥스러운 추신수 "이상하지만, 기분 좋았다"
친구 이대호 앞에서 추신수 첫 안타
[부산 핫포커스] '박수+환호' 홈런 못지 않았던 '첫 안타' 환호…추신수 "기분 이상하던데요"
안철수 “새 잠실야구장, 내년 신축…25개구마다 에어돔” 공약


### 종목 가져오기

In [20]:
menus = driver.find_elements_by_class_name('main_menu_item')
for menu in menus:
    event = menu.find_element_by_tag_name('a').text
    print(event)

스포츠홈
야구
해외야구
축구
해외축구
농구
배구
골프
일반
e스포츠&게임
오늘의 경기
연재
랭킹


In [21]:
menus = driver.find_elements_by_class_name('main_menu_item')
events = []
for menu in menus[1:-3]:
    event = menu.find_element_by_tag_name('a').get_attribute('href')
    events.append(event)
events

['https://sports.news.naver.com/kbaseball/index.nhn',
 'https://sports.news.naver.com/wbaseball/index.nhn',
 'https://sports.news.naver.com/kfootball/index.nhn',
 'https://sports.news.naver.com/wfootball/index.nhn',
 'https://sports.news.naver.com/basketball/index.nhn',
 'https://sports.news.naver.com/volleyball/index.nhn',
 'https://sports.news.naver.com/golf/index.nhn',
 'https://sports.news.naver.com/general/index.nhn',
 'https://sports.news.naver.com/esports/index.nhn']

In [22]:
menus = driver.find_elements_by_class_name('main_menu_item')
events = []
for menu in menus[1:-3]:
    event = menu.find_element_by_tag_name('a').get_attribute('href')
    events.append(event.split('/')[3])
events

['kbaseball',
 'wbaseball',
 'kfootball',
 'wfootball',
 'basketball',
 'volleyball',
 'golf',
 'general',
 'esports']

In [28]:
title_list = []
for event in events:
    print(event, end=': ')
    url = f'{base_url}{event}{sub_url}'
    driver.get(url)
    time.sleep(1)

    while True:
        page = driver.find_element_by_id('_pageList')
        try:
            next = page.find_element_by_class_name('next')
            if next:
                next.click()
        except:
            break;

    page = driver.find_element_by_id('_pageList')
    try:
        atags = page.find_elements_by_tag_name('a')
        numPage = int(atags[-1].text)
    except:
        numPage = 1

    print(numPage, end=' - ')

    for page in range(1, numPage+1):
        print(page, end=', ')
        url = f'{base_url}{event}{sub_url}&page={page}'
        driver.get(url)
        time.sleep(1)

        news = driver.find_element_by_id('_newsList')
        lis = news.find_elements_by_tag_name('li')
        for li in lis:
            text = li.find_element_by_class_name('title')
            title = text.find_element_by_tag_name('span').text
            title_list.append(title)
    print()

kbaseball: 1 - 1, 
wbaseball: 8 - 1, 2, 3, 4, 5, 6, 7, 8, 
kfootball: 11 - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
wfootball: 13 - 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 
basketball: 9 - 1, 2, 3, 4, 5, 6, 7, 8, 9, 
volleyball: 5 - 1, 2, 3, 4, 5, 
golf: 6 - 1, 2, 3, 4, 5, 6, 
general: 8 - 1, 2, 3, 4, 5, 6, 7, 8, 
esports: 9 - 1, 2, 3, 4, 5, 6, 7, 8, 9, 


In [29]:
len(title_list)

1325

In [30]:

title_str = '\n'.join(title for title in title_list)
with open('sports.txt', 'w', encoding='utf-8') as fp:
    fp.write(title_str)